## Description

Resnet Deep Learning model using Keras

## Import Libraries

In [12]:
from sklearn.preprocessing import LabelBinarizer

from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, model_from_json
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.constraints import maxnorm
from keras import regularizers
from keras.layers.normalization import BatchNormalization
from keras.layers import Dense, Dropout, Activation, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from keras.layers import concatenate, add
from keras.models import Model
from keras.regularizers import l2
import keras.backend as K

from keras.applications import imagenet_utils
from keras.preprocessing.image import img_to_array
from keras.callbacks import TensorBoard

import numpy as np 
import json
import os
import cv2
import h5py

import matplotlib.pyplot as plt
%matplotlib inline


ImportError: libcublas.so.10: cannot open shared object file: No such file or directory

In [11]:
from helpers import TrainingMonitor
from helpers import Utils

ImportError: cannot import name 'TrainingMonitor'

In [12]:
output_path = "../output/"

In [15]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)

In [2]:
db_train = h5py.File("../input/datasets/cifar_rgbmean_train.hdf5")
db_test = h5py.File("../input/datasets/cifar_rgbmean_test.hdf5")

x_train_rgbmean = db_train["images"][:].astype('float32')
x_test_rgbmean = db_test["images"][:].astype('float32')

mean = np.mean(x_train_rgbmean, axis=0)
x_train_rgbmean -= mean
x_test_rgbmean -= mean

y_train_rgbmean = db_train["labels"][:]
y_test_rgbmean = db_test["labels"][:]

NameError: name 'h5py' is not defined

In [3]:


class RESNET:
    @staticmethod 
    def residual_module(data, num_filter, stride, red=False, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        shortcut = data
        
        act1 = Activation("relu")(data)
        bn1 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(act1)
        conv1 = Conv2D(int(num_filter * 0.25), (1, 1), use_bias=False, kernel_regularizer=l2(reg))(bn1)
        
        act2 = Activation("relu")(conv1)
        bn2 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(act2)
        conv2 = Conv2D(int(num_filter * 0.25), (3, 3), strides=stride, padding="same", use_bias=False, 
                       kernel_regularizer=l2(reg))(bn2)
        
        bn3 = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(conv2)
        conv3 = Conv2D(num_filter, (1, 1), use_bias=False, kernel_regularizer=l2(reg))(bn3)
        
        if red:
            shortcut = Conv2D(num_filter, 1, strides=stride, use_bias=False, kernel_regularizer=l2(reg))(act1)
                    
        x = add([conv3, shortcut])
        x = Activation("relu")(x)
        x = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(x)
        return x
    
    @staticmethod
    def build(stages, filters, reg=0.0001, bnEps=2e-5, bnMom=0.9):
        input_tensor = Input(shape=x_train_rgbmean.shape[1:])
        
        x = BatchNormalization(axis=-1, epsilon=bnEps, momentum=bnMom)(input_tensor)
        x = Conv2D(filters[0], (3, 3), use_bias=False, padding="same", kernel_regularizer=l2(reg))(x)
        
        for i in range(0, len(stages)): 
            stride = (1, 1) if i == 0 else (2, 2)
            x = RESNET.residual_module(x, filters[i + 1], stride, red=True, bnEps=bnEps, bnMom=bnMom)
            for j in range(0, stages[i] - 1): 
                x = RESNET.residual_module(x, filters[i + 1], (1, 1), bnEps=bnEps, bnMom=bnMom)
        
        x = AveragePooling2D((8, 8))(x)
        x = Flatten()(x)
        x = Dense(10, kernel_regularizer=l2(reg), kernel_constraint=maxnorm(3))(x)
        x = Activation("softmax")(x)
            
        model = Model(input_tensor, x, name="resnet")
        return model



In [4]:
model = RESNET.build((2, 2, 2), (32, 32, 64, 64), reg=0.0005)

NameError: name 'x_train_rgbmean' is not defined

In [5]:
model.summary()

NameError: name 'model' is not defined

In [6]:
for (i, layer) in enumerate(model.layers): 
    print("{}\t{}".format(i, layer.__class__.__name__))
    

NameError: name 'model' is not defined

In [7]:
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
from keras.utils import plot_model
plot_model(model, to_file='models/baseline-resenet.png', show_shapes=True, show_layer_names=True)

NameError: name 'model' is not defined

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

NameError: name 'model' is not defined

In [9]:
history = model.fit(x_train_rgbmean, y_train_rgbmean,
            batch_size=64,
            epochs=10,
            validation_split=0.33,
            shuffle="batch",
            callbacks=callbacks)

NameError: name 'model' is not defined

In [10]:
scores = model.evaluate(x_test_rgbmean, y_test_rgbmean, verbose=0)
print("Train: %.2f%%; Val: %.2f%%; Test: %.2f%%" % 
      (np.max(history.history['acc'])*100, np.max(history.history['val_acc'])*100, scores[1]*100)
     )


NameError: name 'model' is not defined

In [11]:
model_json = model.to_json()
with open(output_path + "cifar_model_resnet_94.02.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights(output_path + "cifar_weight_resnet_94.02.hdf5")

NameError: name 'model' is not defined